# Desafio Final Imersão Alura

## Introdução


Drug Discover [1] pode ser descrita como o processo de identificação de entidades químicas com potencial para se tornarem agentes terapêuticos. Um objetivo principal das campanhas de descoberta de medicamentos é o reconhecimento de novas entidades moleculares que podem ser valiosas no tratamento de doenças que se qualificam como apresentando necessidades médicas não atendidas. Essas doenças não têm terapias definitivamente úteis e são real ou potencialmente fatais. Os medicamentos comercializados neste momento representam um número relativamente pequeno de tipos-alvo de medicamentos. Os medicamentos direcionados contra os receptores acoplados à proteína G, os receptores nucleares (hormonais) e os canais iônicos representam um pouco menos de 50% dos medicamentos comercializados. De longe, os medicamentos dirigidos contra enzimas representam a maior parte dos medicamentos comercializados. A expansão para novos tipos de alvos de drogas pode ser necessária para preencher certos vazios terapêuticos, mas uma questão de grande desafio intelectual é como escolher um alvo que provavelmente tenha valor, especialmente ao se aventurar em tipos menos explorados de alvos de drogas.

O processo de pesquisa e desenvolvimento farmacêutico tradicional sofre com uma alta taxa de desgaste. Para cada nova droga lançada no mercado, a maioria das estimativas sugere que os pesquisadores normalmente terão empregado mais de 100 triagens procurando pistas de drogas, separando candidatos de dezenas de milhares de compostos, conforme a figura mostrada abaixo.

<center><img src="https://ars.els-cdn.com/content/image/3-s2.0-B9780123854711000301-f30-01-9780123854711.jpg" align="center"></center>

E é neste contexto que a Ciência de Dados se faz presente. Com ela podemos economizar recursos e agilizar o processo através da automação para que novos compostos possam ser identificados, podendo tornar-se remédio.




## Importações

In [96]:
import pandas               as pd
import numpy                as np
import matplotlib.pyplot    as plt
import seaborn              as sns


from random                 import sample




import warnings


In [97]:
warnings.filterwarnings('ignore')

Os dados utilizados neste projeto foram fornecido pela Alura durante a Imersão Dados. Os dados podem ser encontrados neste [link](https://github.com/alura-cursos/imersao-dados-desafio-final/tree/main/Dados).

A origem dos dados é da competição *Mechanisms of Action (MoA) Prediction* que pode ser encontrada no site Kaggle, fornecido pelo Connectivity Map, um projeto da Broad Institute of MIT e Harvard, do LISH (Laboratory for Innovation Science at Harvard) e LINCS (NIH Common Funds Library of Integrated Network-Based Cellular Signatures). O objetivo desta competição é promover o avanço na área de drug development por meio de algoritmos de aprendizado de máquinas.

Para acessar a página da competição clique [aqui](https://www.kaggle.com/c/lish-moa)

Os arquivos fornecidos foram 2 datasets:

1. **dados_experimentos.zip** - Contém os experimentos e dados referentes a sua observação de acordo com as variáveis

| Coluna | Descrição | Valor |
| ------ | --------- | ----- |
|id| Código para identificar o experimento| `str` contendo números e letras|
|tratamento| A amostra é do grupo controle ou é um<br>experimento| `com_controle` = grupo controle <br> `com_droga` = Experimento |
|dose| Dose da droga Utilizada| `D1` = dose anônima *<br>`D2` = dose anônima *|
|tempo| Tempo decorrido do início ao realizar a<br>observação| `24` = 24 horas <br>`48` = 48 horas<br>`72` = 72 horas|
|droga| Composto utilizado no experimento |`str` contendo números e letras|
|g-x| Expressão Gênica| `float`|
|c-x| Viabilidade Celular| `float`|

2. **dados_resultados.csv** - Dataframe com o resultados para cada experimento do dataset `dados_experimentos.zip` com os mecanismos de ação ativados. A coluna `id` relaciona os experimentos aos resultados

In [ ]:
dados = pd.read_csv('https://github.com/alura-cursos/imersaodados3/blob/main/dados/dados_experimentos.zip?raw=true', compression='zip')
dados_resultados = pd.read_csv('https://github.com/alura-cursos/imersaodados3/blob/main/dados/dados_resultados.csv?raw=true')

O primeiro passo antes de analisarmos o dataset é verificar se há algum problema com as bases de dados

In [ ]:
print('dados_experimentos.zip\n')
print(f'Quantidade de Linhas = {dados.shape[0]}')
print(f'Quantidade de Colunas = {dados.shape[1]}\n')

print('====================\n')

print('dados_resultados.csv\n')
print(f'Quantidade de Linhas = {dados_resultados.shape[0]}')
print(f'Quantidade de Colunas = {dados_resultados.shape[1]}')

In [ ]:
# Pega a coluna id do 2 datasets
id_dados = dados['id'].unique()
id_dados_resultados = dados_resultados['id'].unique()

# Verifica se há algum id diferente
if all(id_dados == id_dados_resultados):
    print('Todos os ids tem seu correspondente no outro dataset')
else:
    print('Há uma inconsistẽncia no dataset. Verifique!')

A primeira verificação é para garantir que o número de linhas entre os dois datasets sao iguais.

A segunda verificação era para garantir que um id tem seu correnpondente no outro dataset

Todos as verificações se mostraram verdadeiras. Agora, será que há algum valor `NaN` (Not a Number) em nossos datasets?

Vamos verificar!

In [ ]:
# Para dados_experimentais.zip
if all(dados.isna().sum() != 0):
    print('Há valor NaN')
else:
    print('Não valor NaN em dados_experimentais.zip')

# Para dados_resultados.csv
if all(dados_resultados.isna().sum() != 0):
    print('Há valor NaN')
else:
    print('Não valor NaN em dados_resultados.csv')
    

Agora podemos analisar as colunas categóricas de `dados_experimental.zip` (tratamento, tempo, dose, droga)

In [ ]:
colunas_categoricas = ['tratamento', 'tempo', 'dose', 'droga']

for coluna in colunas_categoricas:
    print(f'A coluna {coluna} possui {dados[coluna].nunique() } categorias')

Para iniciar a exploração dos dados. Que tal colocarmos uma lupa em cada uma destas variáveis?

Vamos lá?

## Análise Exploratória dos Dados

### Tratamento

#### Podemos dizer que o tipo de tratamento está dividido exatamente igual?

In [ ]:
sns.set(rc={'axes.facecolor':'f6f5f5', 'figure.facecolor':'f6f5f5'})

In [ ]:
x = dados['tratamento'].value_counts()

fig, ax = plt.subplots(figsize = (6,6), dpi = 70)


sns.barplot(x=x.values, y=x.index, ax=ax)
plt.text(x=-4150, y=0.00, s='Droga', fontdict={'family': 'Serif',
                                                  'weight':'bold',
                                                  'Size': '16',
                                                  'style':'normal',
                                                  'color':'#512b58'})

plt.text(x=-5800, y=1.00, s='Controle', fontdict={'family': 'Serif',
                                                  'weight':'bold',
                                                  'Size': '16',
                                                  'style':'normal',
                                                  'color':'#512b58'})

plt.text(x=-6150, y=-0.88,
         s='Porcentagem de Experimentos',
         fontdict={'family': 'Serif',
                   'Size': '25',
                   'weight':'bold',
                   'color':'black'})

plt.text(x=-6000, y=-0.63,
         s='Há um grande desbalanceamento entre os experimentos \ncontrole e experimentos que utilizam drogas', 
        fontdict={'family':'Serif', 'size':'17.5','color': 'black'})


plt.text(x=22500, y=0.00, s='92%', fontdict={'family': 'Serif',
                                            'weight':'bold',
                                            'Size': '16',
                                            'style':'normal',
                                            'color':'#512b58'})

plt.text(x=2800, y=1.00, s='8%', fontdict={'family': 'Serif',
                                          'weight':'bold',
                                          'Size': '16',
                                          'style':'normal',
                                          'color':'#512b58'})



ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(True)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

Neste dataset analisado, podemos ver que temos mais experimentos utilizando drogas do que como controle.

A explicação para isto é que o grupo controle pode servir para diversos experimentos que utilizam drogas.

O grupo de controle permite que visa isolar apenas uma variavel do experimento. Em nosso caso, o grupo controle éstá sob as mesmas condições de conservação, exceto que não recebeu a droga.

Um grupo de controle científico nos dá a possibilidade de avaliar uma variável por vez. Ao desenharmos os experimentos, submetemos um grupo as condições normais de funcionamento e um outro grupo mudamos apenas um aspecto. Para o primeiro grupo nós damos o nome de **grupo de controle** e para o segundo **estudo experimental**.

Cabe ressaltar que não é necessário experimentos em pares, podemos testar diferentes compostos sob as mesmas condições de um único grupo de controle

#### O que é colocado na coluna `droga` nas amostra do grupo de controle?

In [ ]:
dados_controle = dados[dados['tratamento'] == 'com_controle']
print(dados_controle['droga'].unique())

#### Se a droga utilizada é a `cacb2b860` ela tem dose diferentes?

In [ ]:
dados_controle['dose'].value_counts()

O grupo de controle também são divididos de acordo com a dose?

Você pode me perguntar como o grupo de controle pode ter dose se ele é exatamente a cultura de bacterias na mesma condições do experimento?

E é exatamente por essa pergunta que podemos inferir que *TALVEZ* esta droga aplicada no grupo de controle seja um placebo, algo que não ative os mecanismos de ação das amostras. 

### Tempo

#### E a variável `tempo`? É balanceada? 

In [ ]:
sns.countplot(data=dados, y='tempo')

A variável `tempo` está balanceada. Bem distribuída entre as 3 categorias (24, 48, 72). Esta observação tem sentido, pois se o experimento foi desenhado para fazer observações nestes momentos, espera-se que mais ou menos todas análises tenham sido feitas nos tempos marcados.

A feature `tempo` é tempo decorrido desde a aplicação do composto até a observação. Então se a label na variável `tempo` é 24, aqueles dados refere-se à observação 24 horas depois da aplicação do composto.

#### Eu vejo que ela não tem exatamente o mesmo número de observações o que é isso?

### Dose

#### Como está distribuída a variável `dose`?

In [ ]:
sns.countplot(data=dados, x='dose')

A variável `dose` refere-se a dose aplicada do composto no experimento.

O motivo de não revelarem qual dose é a maior é puramente para a eliminação de viés por quem analisa os dados. Este comportamento, de anonimizar os dados, está presente por todo dataset,

Uma outra análise que podemos fazer é sobre o número de composto da base de dados

### Drogas

#### Quantas drogas há em nosso dataset?


In [ ]:
qnt_drogas = dados['droga'].nunique()
print(f'Há {qnt_drogas} em nosso dataset')

#### Quais são as 5 drogas mais utilizadas?

In [ ]:
contagem_droga = dados['droga'].value_counts().sort_values(ascending=False)[1:6]
sns.barplot(y=contagem_droga.index, x=contagem_droga.values)

Em nosso dataset, a droga mais utilizada é o placebo utilizado no grupo de controle. Ao retirarmos este rótulo, nosso Top 5 fica

1. `87d714366`
2. `9f80f3f77`
3. `8b87a7a83`
4. `5628cb3ee`
5. `d08af5d4b`

Aposto que você está se perguntando quais são estas drogas?

Mais uma vez aquele padrão de anonimizar os dados para evitar viés por parte de quem os analisa.

#### As mesmas drogas ativam mecanismos de ação diferentes?

### Mecanismos de Ação

In [ ]:
dados_resultados['moa_ativados'] = ''

for col_name in dados_resultados.columns:
    dados_resultados.loc[dados_resultados[col_name] == 1, 'moa_ativados'] = dados_resultados['moa_ativados'] + ' | ' + col_name

dados_resultados['moa_ativados'] = dados_resultados['moa_ativados'].str.lstrip('| ')

In [ ]:
dados_resultados['n_moa'] = dados_resultados.drop('id', axis=1).sum(axis=1)
dados_resultados['ativo_moa'] = (dados_resultados['n_moa'] != 0)
dados_resultados.head()

#### Quantos Mecanismos de ação há em nosso dataset

In [ ]:
qnt_moa = len(dados_resultados.columns) - 4
print(f'Há {qnt_moa} em nosso dataset')

#### Como estão distribuídas o número de MoA?

In [ ]:
qnt_cat_n_moa = dados_resultados['n_moa'].value_counts().sort_values(ascending=False)
qnt_cat_n_moa = qnt_cat_n_moa.reset_index()
qnt_cat_n_moa.columns = ['cat_n_moa', 'quantidades']

quantidade_2_ou_mais = qnt_cat_n_moa.loc[2:7]['quantidades'].sum()

df_cat_2_ou_mais = pd.DataFrame([['2 ou mais', quantidade_2_ou_mais]],
                                columns=['cat_n_moa', 'quantidades'])

qnt_cat_n_moa = qnt_cat_n_moa.drop([2,3,4,5,6]).append(df_cat_2_ou_mais, ignore_index=True)

sns.barplot(data=qnt_cat_n_moa, y='quantidades', x='cat_n_moa')

In [ ]:
dados_resultados['n_moa'] = dados_resultados.drop('id', axis=1).sum(axis=1)
dados_resultados['ativo_moa'] = (dados_resultados['n_moa'] != 0)
dados_resultados.head()

#### Qual a relação entre experimento com MoA ativados e não ativados?

In [ ]:
round(dados_resultados['ativo_moa'].value_counts(normalize=True),2)

#### Qual o MoA mais ativados?

In [ ]:
qnt_moa = dados_resultados.drop(['id', 'n_moa', 'ativo_moa', 'moa_ativados'],
                                axis=1
                                ).sum(axis=0).sort_values(ascending=False)[:10]

sns.barplot(y=qnt_moa.index, x=qnt_moa.values)

#### Uma droga pode ativar diversos tipos de MoA?

In [ ]:
lista_drogas = list(dados['droga'].unique())

flag = False

for droga in lista_drogas:

    lista_id = list(dados[dados['droga'] == droga]['id'].values)

    moa_ativos = dados_resultados[dados_resultados['id'].isin(lista_id)]['moa_ativados'].unique()
    
    if len(moa_ativos) > 1:
        print(f'A droga {droga} ativa diferentes combinações de Moa')
        flag = True

if not flag:
    print('Drogas ativam as mesma combinações de MoA')

#### Qual é a relação entre os tipos de MoA presente no dataset?

Ao examinarmos o dataset `dados_resultados.csv` podemos ver que a maioria das colunas terminam com as seguintes palavras

1. `_inhibidor`
2. `_antagonist`
3. `_agonist`

Há MoA com outras terminações. E também há moa que não tem nem estas terminações.

O primeiro passo é entender o que significa esses termos. Isso é uma área chamada de *farmacodinâmica* (estudo dos efeitos de um composto/droga no corpo)

O estudo desta área baseia-se no conceito da ligação fármaco-receptor.


Um composto pode se ligar a uma célula e pode ocorre uma resposta como consequência desta ligação. Tudo depende da chamada curva de *dose x resposta*

Muitos receptores de compostos podem ser categorizados dentro de dois estados de conformação, que estão em equilíbrio reversível entre si. Temos com grande divisão:

- **Agonista** - Um composto que, através de sua ligação a seu receptor, favorece a conformação ativa deste receptor é denominado agonista

- **Antagonista** - Um fármaco que impede a ativação do receptor pelo agonista é designado como antagonista.

**Em resumo, agonista e antagonista são duas pessoas no aniversário Guanabara disputando a última caixa de geléia de mocotó R$0,16 mais barato.O mais forte, leva.** 😅

Por exemplo:

`retinoid_receptor_agonist`, `retinoid_receptor_antagonist` competem entre si para pelo receptor de retinoides, moléculas quimicamente relacionadas com a vitamina A

In [ ]:
def remove_items(lista, item):
      
    lista_nova = [i for i in lista if i != item]
  
    return lista_nova

def get_cat_moa(df, value):

    moas = df['moa_ativados']
    lista_moas = moas.split(' | ')
    acao = [j.split('_')[-1] for j in lista_moas]

    if value == 'others':

        acao = remove_items(acao, 'inhibitor')
        acao = remove_items(acao, 'agonist')
        acao = remove_items(acao, 'antagonist')

        if (len(acao) == 1) and ('' in acao):
            return 0

        if len(acao) > 0:
            return 1


    if value in acao:
        return 1
    return 0

In [ ]:
dados_resultados['inibidor'] = dados_resultados.apply(get_cat_moa, value='inhibitor', axis = 1)
dados_resultados['agonista'] = dados_resultados.apply(get_cat_moa, value='agonist', axis = 1)
dados_resultados['antagonista'] = dados_resultados.apply(get_cat_moa, value='antagonist', axis = 1)
dados_resultados['outros'] = dados_resultados.apply(get_cat_moa, value='others', axis = 1)

In [ ]:
cat_terminacoes = dados_resultados[['inibidor', 'agonista', 'antagonista', 'outros']].sum().sort_values(ascending=False)

sns.barplot(x=cat_terminacoes.values, y=cat_terminacoes.index, orient='h', )

### Distribuição da Expressão Gênica

In [ ]:
g_x = [f'g-{i}' for i in range(772)]

g_random = dados[sample(g_x, 4)]

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(16,9))

ax = ax.ravel()

for i, df in enumerate(g_random):
    sns.histplot(data=dados[df], multiple='stack', kde=True, ax=ax[i])

### Distribuição da Viabilidade da Célula

In [ ]:

c_x = [f'c-{i}' for i in range(100)]

c_random = dados[sample(c_x, 4)]

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(16,9))

ax = ax.ravel()

for i, df in enumerate(c_random):
    sns.histplot(data=dados[df], multiple='stack', kde=True, ax=ax[i])

Com uma análise do gráfico, podemos ver que as colunas C-X e G-X tem uma distribuição normal. Tudo nos leva a crer que a essas variáveis passaram por uma transformação que alterou sua distribuição.

Vamos ver a correlação entre elas

### Correlação entre as variáveis númericas

#### G-x's

In [ ]:
corr = dados.loc[:,'g-0':'g-50'].corr()

In [ ]:
%matplotlib inline
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(20, 20))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(250, 100, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()

Podemos que a maioria das combinações entre os g's fica com cor de burro quando foge (correlação próxima de 0). 

O genes com a maior correlação é o g-50 com g-37

#### C-x's

In [ ]:
corr = dados.loc[:,'c-0':'c-50'].corr()

In [ ]:
%matplotlib inline
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(20, 20))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(250, 100, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()

Entre as variáveis C's vemos que há um alta correlação entre elas.

*Pra eu que já não enxergo cor muito bem, então*!

## Analisando um MoA

Resolvi analisar o ativamento do MoA `serotonin_receptor_antagonist`.

Você pode se perguntar: "Porque esse MoA?"

Sendo sincero, após uma pesquisa, vi que esse é Mecanismo de Ação do Vonau Flash, um remédio jaboticaba, feito no Brasil, pela USP. A bula que diz sobre o mecanismo de ação pode ser acessada [aqui](https://consultaremedios.com.br/vonau-flash/bula)

Pela pouca pesquisa que fiz, é um remédio que leva vantagem sobre os concorrentes é que no é preciso engolir, ele dissolve na boa. Excelente opção para crianças, ou a pessoa que vos escreve, que acham ruim engolir o remédio

Outra vantagem é que ele age mais rápido que todos os outro concorrentes.


Em 2018, a patente respondia por **58% de toda a receita da USP com royalties de invenções.**

### serotonin_receptor_antagonist

Primeiro ver a quantidade de ativação deste MoA

In [ ]:
dados_combinados = pd.merge(dados, dados_resultados[['id', 'serotonin_receptor_antagonist']], on='id')
dados_combinados

In [ ]:
print(round(dados_combinados['serotonin_receptor_antagonist'].value_counts(normalize=True),2))
sns.countplot(data=dados_combinados, y='serotonin_receptor_antagonist')

Apenas 2% dos dados totais ativaram o Moa `serotonin_receptor_antagonist`.]
Se pensarmos em fazer um modelo, necessitamos de um balanceamento deste target

### Quais drogas estão associados à `serotonin_receptor_antagonist`?

In [ ]:
dados_serotonin = dados_combinados[dados_combinados['serotonin_receptor_antagonist'] == 1]
dados_serotonin['droga'].nunique()

Há **68** drogas relacionadas à este MoA. Outra pergunta que nos permeia é...

### Quantidades por droga?

In [ ]:
dados_combinados[dados_combinados['serotonin_receptor_antagonist'] == 1]['droga'].value_counts().sort_values(ascending=False)

### É ativada com outras MoA?

In [ ]:
id_dados_serotonin = list(dados_serotonin['id'].values)

In [ ]:
moa_ativados_serotonin = dados_resultados[dados_resultados['id'].isin(id_dados_serotonin)]
moa_ativados_serotonin['moa_ativados'].nunique()

O `serotonin_receptor_antagonist` ativa em mais 9 combinações de MoA.

In [ ]:
for combinacao in moa_ativados_serotonin['moa_ativados'].unique():
    print(combinacao)
    print()

Estas são as combinações de MoA em que o `serotonin_receptor_antagonist` participa

# Qual droga ativa apenas o `serotonin_receptor_antagonist`?


In [ ]:
lista = list(moa_ativados_serotonin[moa_ativados_serotonin['moa_ativados'] == 'serotonin_receptor_antagonist']['id'].values)

In [ ]:
dados_combinados[dados_combinados['id'].isin(lista)]['droga'].unique()

Todos as drogas acime PODE ser um composto que está dentro do Vonal Flash. *Considerando que o remédio ativa apenas um MOA*

## Conclusões

Atravé do projeto apresentado pela Alura conseguimos identificar e analisar Mecanismo de Ação que está dentro do Vonal Flash

## Próximo Passos

Como Próximos Passos podemos criar um modelo que preveja quando o MoA `serotonin_receptor_antagonist` ative

## Referências

[1] [Drug Discovery](https://www.sciencedirect.com/sdfe/pdf/download/eid/3-s2.0-B9780123854711000301/first-page-pdf)

[2] https://anestesiologia.paginas.ufsc.br/files/2015/02/Farmacodinamica-texto.pdf

[3] https://pt.wikipedia.org/wiki/Retinoides

[4] https://exame.com/ciencia/mais-que-ciencia-a-gestao-do-vonau-flash-patente-milionaria-da-usp/



